# [0. Settings](#0) 
<br></br>
## [0.1 Librerie](#0.1)

<font size=3>

Le librerie importate hanno le seguenti funzionalità:
* `selenium`: la libreria principale attraverso la quale si effettuerà lo scraping. Da esse sono importati diversi moduli:
    * `webdriver`: attraverso `selenium` è possibile tramite codice aprire una nuova finestra del browser. Nel caso in questione è stato utilizzato Firefox, la comunizazione fra i due avviene attraverso un file esterno `geckodriver.exe`.
    * `ActionChains`: permette al browser di simulare azioni di un utente come ad esempio un click.
    * `NoSuchElementException`: nella fase di scraping si cercherà di individuare elementi tramite il loro `xpath`, nel caso l'elemento non sia individuato l'errore risultate è quello riportato. Per non bloccare il codice davanti a questi errori ma lasciarlo continuare è necessario importare questa liberia.
    * `ElementNotInteractableException`: come il precedente ma è riferito ad un elemento con cui non si può interagire (ad esempio un click).
* `Counter`: una funzione che permette di contare gli elementi all'interno di una lista o un dizionario e ne restituisce la frequenza. Utile per controllare se ci sono dei duplicati.
* `deepflatten`: permette di trasformare liste di liste in un'unica lista.
* `os`: permette di specificare un percorso in locale da cui accedere a dei file.
* `pandas`: libreria per gestire dataframe.
* `pickle`: permette di salvare elementi creati in una sessione e caricarli in una successiva senza alternarne la loro struttura.
* `time`: serve per gestire la tempistica con cui le operazioni di scraping verranno effettuate.
* `tqdm`: se applicato ad un ciclo stima il tempo necessario per tutte le iterazioni.

In [20]:
from collections import Counter
from iteration_utilities import deepflatten

from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException

from tqdm import tqdm

import pandas as pd
import pickle
import os
import time

os.chdir('D:/Scraping')

<br></br>
## [0.2 Impostazioni Firefox](#0.2)

<font size=3>

Nelle impostazioni del browser vengono disabilitate le immagini, questo permette un caricamento della pagina più veloce e allo stesso tempo non modifica il codice degli elementi da invididuare.

In [2]:
firefox_profile = webdriver.FirefoxProfile()
firefox_profile.set_preference('permissions.default.image', 2)
firefox_profile.set_preference('dom.ipc.plugins.enabled.libflashplayer.so', 'false')

<br></br>
## [0.3 Pickle Load](#0.3)

<font size=3>

L'operazione di scraping è stata effettuata in più sessioni per via dei tempi esigenti nell'ottenere tante recensioni di uno stesso film, da qui l'esigenza di creare dei file `pickle` che permettono di salvare un oggetto e di ricaricarlo senza che la sua struttura sia modificata, in questo caso vengono caricate delle liste.

In [5]:
with open('output_review', "rb") as input_file:
    review_list = pickle.load(input_file)
with open('output_score', "rb") as input_file2:
    score_list = pickle.load(input_file2)
with open('output_date', "rb") as input_file3:
    date_list = pickle.load(input_file3)
with open('output_title', "rb") as input_file4:
    title_list = pickle.load(input_file4)

<br></br> <br></br>
# [1. Scraping](#1)
<br></br>
## [1.1 Funzione recensioni](#1.1)

<font size=3>
    
La funzione `reviews` permette di estrarre le informazioni da un film. Prende in input i seguenti parameri:
* `url`: un indirizzo da quale sono mostrati i film di interesse. Nel caso in questione i film da cui trarre informazioni sono film Drama, preferibilmente con una quantità elevata di commenti. L'url può quindi essere scomposto nelle seguenti parti: <br></br>
    * `base`: <i>"https://www.imdb.com/search/title/?title_type=feature"</i>, è la parte comune a tutti gli url, viene ricavato dalla funzione di ricerca avanzata del sito.
    * `data`: <i>"&release_date=YYYY-MM-DD,YYYY-MM-DD"</i>, l'approccio prevede l'estrazione dei commenti di film usciti in anni diversi, nella fase di ricerca è possibile indicare un filtro con cui sono selezionati i film di un determinato anno. Nel primo parametro viene specificata la data minima di uscita del film, $1$ gennaio dell'anno specifico, mentre nel secondo la data massima, $31$ dicembre dell'anno specifico.
    * `genere`: <i>"&genres='genre'"</i>, il genere specifico da cui estrarre i film, nel caso in questione è stato scelto "drama" in quanto offre il numero più elevato di film e di conseguenza di commenti.
    * `numero risultati`: <i>"&count='num'"</i>, rappresenta il numero di risultati da visualizzare per pagina, per lo scraping è stato posto al massimo ottenibile dal sito, ovvero $250$. Questo rende le operazioni di scraping più semplici, in quanto dopo il $250$&ndash;esimo film il numero di recensioni inzia ad essere esiguo e quindi non conviene estrarre ulteriori commenti, in questo modo si evita di caricare la pagina successiva, un passaggio delicato che potrebbe portare a degli errori.
    
* `review_list`: in un blocco a parte viene inizializzata una lista in cui verrano inserite le recensioni. Il risultato sarà una lista di liste dove ogni lista interna conterrà un numero variabile di commenti ma sempre riferiti allo stesso film.
* `score_list`: come sopra in riferimento al punteggio della recensione su una scala da $1$ a $10$.
* `date_list`: come sopra in riferimento alla data della recensione.
* `title_list`: una lista in cui viene salvato il nome del film. A differenza delle tre liste precedenti, questa è una lista semplice in quanto, come dice il nome, è in riferimento al film e non alla singola recensione.

<br></br>
Il codice inzia definendo la variabile `movie` che apre il browser sul'url predefinito; in questa pagina viene visualizzata una lista dei $250$ film drama più popolari di un anno specifico. Il ciclo `for`  si sviluppa per un numero grande tanto quanto sono i film per pagina. All'interno di esso sono effettuate le seguenti operazioni:
* Viene identificato l'`xpath` relativo al titolo del film, nel caso specifico il titolo viene suddiviso dal sito in due elementi distinti, per questo vengono inzializzate due variabili, `title` e `title 2`, che fanno riferimento a due xpath diversi. Il motivo per cui il sito effettua questa distinzione è per evitare casi di omonimia: la prima parte è il titolo vero e proprio del film, la seconda parte è solamente l'anno di uscita messo tra parentesi. <br></br> 
L'oggetto ricavato è nel formato della libreria utilizzata, in questo caso `selenium`, per accedere al contenuto testuale si deve applicare la funzione `text` alle variabili. I due titoli vengono uniti mediante un separatore come spazio e il risultato è aggiunto alla lista dei titoli. <br></br>
La pagina riporta i film come un elenco, quindi per selezionare di volta in volta il film successivo è necessario modificare l'xpath ad ogni iterazione, da qui l'esigenza di introdurre l'elemento `str(i + 1)` in prossimità della parte che fa riferimento all'ordine nella lista dei film.
* Sono inizializzate delle liste temporanee per recensioni, punteggi e date. In queste liste verranno salvate informazioni per ogni recensione e delle nuove liste vengono inizializzate quando si passa al film successivo. In questo modo si creano liste di liste in cui ogni lista contiene informazioni relative ad un unico film.
* Viene catturato un nuovo xpath, in questo caso il collegamento ipertestuale che porta alla pagina principale del film. Ancora una volta è necessario chiamare una funzione, `get_attribute`, per ricavare il valore testuale identificato. Si effettua una modifica all'url in modo tale da accedere alla pagina delle recensioni. Qui viene aperta una nuova finestra per accedere alle recensioni.

Una complicazione nel processo di ricavare le recensioni riguarda la possibilità di visualizzare a video le recensioni stesse. Di default il sito permette di visualizzare solamente $25$ recensioni e questa quantità non è modificabile inserendo degli elementi aggiuntivi nell'url. L'unica via percorribile è la seguente:
* In fondo alla pagine è presente la casella `Load More` che se cliccata permette di visualizzare ulteriori 25 recensioni. 
* Viene quindi identificato l'xpath corrispondente a quella casella e tramite la funzione `click` è possibile in maniera automatico cliccare sull'elemento trovato.
* Le recensioni di un film possono raggiungere, nel caso di film molto popolari, anche oltre $5000$ unità, la navigazione sulla pagina diventa estremamente lenta per la quantità eccessiva di testo, da qui l'esigenza di limitare il numero di possibili click a $39$ per un massimo di $1000$ commenti per film. 
* È necessario effettuare un cooldown fra un click e l'altro in modo da lasciare abbastanza tempo al browser di caricare i nuovi commenti. Senza il comando `time.sleep` che permette di fermare il codice per un certo lasso di tempo, nel caso in questione $2$ sono sufficienti, il codice una volta caricati i nuovi commenti cercherebbe immediatamente la casella "Load More", ma in alcuni casi una ricerca così repentina porta ad un errore di elemento non trovato.
* Per i film meno conosciuti invece si pone il problema opposto, ovvero che non ci sono molti commenti e quindi non è presente la casella "Load More" direttamente all'inizio o prima delle iterazioni prefeissate. A questo proposito è stato creato un blocco di `try` &ndash; `except`, nel caso in cui l'elemento da cliccare non fosse disponibile, il codice passa direttamente allo step successivo.

Caricate tutte le recensioni possibili, esse si trovano in un formato simile ad una lista, come capitava per i titoli dei film. Tutti gli xpath che si identificheranno dovranno quindi essere modificati con `str(i + 1)` per ricavare di volta in volta la recensione di interesse. Questo ultimo step si articola in:
* Un ciclo for che identifica $25$ elementi tante volte quanti sono stati i click effettuati in precedenza.
* Di default le recensioni lunghe risultano trocante, con una freccia in basso a destra che se cliccata permette di visualizzare il testo completo. Per ogni recensione si verifica l'eventuale esistenza di questo elemento, sempre facendo riferimento all'xpath relativo, se è trovato allora si applica un click per visualizzare a schermo il commento completo.
* Esiste un'altra categoria di recensioni che sono quelle contrassegnate da spoiler, per queste non è possibile vedere alcuna porzione di testo e si deve cliccare sempre una freccia in basso a destra per mostrare il tutto. Si è dovuto implementare un blocco di `try` &ndash; `except` aggiuntivo in quanto gli xpath di riferimento tra le recensioni troncate e spoiler sono differenti.
* Se non risultano problemi con l'xpath della recensione, il contenuto viene salvato nella lista temporanea delle recensioni così come per la data e lo score.
* Una volta raccolte tutte le informazioni delle recensioni di uno specifico film, le liste temporanee vengono salvate all'interno delle liste passate come parametri della funzione.

Quando tutte le informazioni di un film sono state salvate, la finestra del browser con le recensioni viene chiusa e si inizia una nuova iterazione per un nuovo film. Al termine di tutti i $250$ film, viene chiuso anche la finestra iniziale con la lista di tutti i film e i contenuti delle liste sono messi in output dalla funzione. <br>
Si è ritenuto necessario aprire e chiudere le finestre del browser ad ogni iterazione in quanto proseguire con i comandi di pagina precedente, è noto che la velocità del browser diminuisce all'aumentare delle iterazioni.

In [36]:
def reviews(url, review_list, score_list, date_list, title_list):
    movie = webdriver.Firefox(executable_path=r'D:/Scraping2/geckodriver.exe')
    movie.get(url)
    
    for i in range(250):
        """
        Nella lista verranno salvate le recensioni di un singolo film, questa lista verrà aggiunta alla lista creata prima.
        Dall'xpath si estrae l'elemento 'href' che contiene l'indirizzo url della pagina del film.
        """
        title = movie.find_element_by_xpath('/html/body/div[3]/div/div[2]/div[3]/div[1]/div/div[3]/div/div[' + str(i + 1) + ']/div[3]/h3/a')
        title2 = movie.find_element_by_xpath('/html/body/div[3]/div/div[2]/div[3]/div[1]/div/div[3]/div/div[' + str(i + 1) + ']/div[3]/h3/span[2]')
        title_list.append(title.text + ' ' + title2.text)
        
        temp_review_list = [] 
        temp_score_list = []
        temp_date_list = []
        xpath = '//*[@id="main"]/div/div[3]/div/div[' + str(i + 1) + ']/div[3]/h3/a'
        element = movie.find_element_by_xpath(xpath=xpath)
        href = element.get_attribute('href')

        temp_url = href.split('?')
        reviews_url = temp_url[0] + "reviews" # Si crea l'url per le recensioni nascondendo quelle con spoiler
        movie2 = webdriver.Firefox(executable_path=r'D:/Scraping2/geckodriver.exe')
        movie2.get(reviews_url)
        
        
        """
        Si caricano le recensioni e si clicca su "Load more" un numero predefinito di volte (10 nell'esempio).
        Ogni click fa apparire 25 nuove recensioni, se non ce ne sono abbastanza, l'exception fa andare alla parte successiva.
        Il click avviene ogni 3 secondi, senza un delay il browser poteva non identificare l'elemento anche se presente.
        """
        next_review = 0
        click_count = 0
        while click_count < 39:
            try:
                load_more = movie2.find_element_by_xpath(xpath='//*[@id="load-more-trigger"]')
                load_more.click()
                click_count = click_count + 1
                time.sleep(2)
            except (NoSuchElementException, ElementNotInteractableException):
                click_count = 39
                
        """
        Si identifica l'xpath relativo ad ogni recensione, se non è trovato si passa avanti.
        Per ogni recensione si verifica se è troncata e bisogna cliccare la freccia per far comparire tutto il testo.
        Ad ogni iterazione la recensione viene salvata nella lista interna.
        A fine iterazioni la lista creata viene aggiunta alla lista principale e la finestra viene chiusa.
        Da qui poi si prende l'url del film successivo e si procede per 50 film.
        """
        for i in range(25 * click_count):
            review_xpath = '/html/body/div[3]/div/div[2]/div[3]/div[1]/section/div[2]/div[2]/div[' + str(i + 1) + ']/div[1]/div[1]/div[3]/div'
            try: # verifica se la recensione è troncata
                long_text = movie2.find_element_by_xpath('/html/body/div[3]/div/div[2]/div[3]/div[1]/section/div[2]/div[2]/div[' + str(i + 1) + ']/div[2]/div[2]/div[1]/div/div')
                long_text.click()
            except (NoSuchElementException, ElementNotInteractableException):
                try: # prova a vedere se la recensione è uno spoiler
                    long_text = movie2.find_element_by_xpath('/html/body/div[3]/div/div[2]/div[3]/div[1]/section/div[2]/div[2]/div[' + str(i + 1) + ']/div/div[1]/div[2]/div/div')
                    long_text.click()
                    review_xpath = '/html/body/div[3]/div/div[2]/div[3]/div[1]/section/div[2]/div[2]/div[' + str(i + 1) + ']/div/div[1]/div[4]/div[1]'
                except (NoSuchElementException, ElementNotInteractableException):
                    pass

            try: # verifica se esiste effettivamente la recensione
                review = movie2.find_element_by_xpath(review_xpath)
                temp_review_list.append(review.text)

                score = movie2.find_element_by_xpath('/html/body/div[3]/div/div[2]/div[3]/div[1]/section/div[2]/div[2]/div[' + str(i + 1) + ']/div[1]/div[1]/div[1]/span/span[1]')
                temp_score_list.append(score.text)

                date = movie2.find_element_by_xpath('/html/body/div[3]/div/div[2]/div[3]/div[1]/section/div[2]/div[2]/div[' + str(i + 1) + ']/div[1]/div[1]/div[2]/span[2]')
                temp_date_list.append(date.text)

            except (NoSuchElementException):
                pass

        review_list.append(temp_review_list)
        score_list.append(temp_score_list)
        date_list.append(temp_date_list)
        movie2.quit()
        
    movie.quit()
    
    return (review_list, score_list, date_list, title_list)

<font size=3>

Nella seguente riga di codice sono create le liste che saranno passate come input alla funzione di scraping. Le liste sono inizializzate una volta sola e non per ogni anno, essendo soggette solamente ad un append, anche passando alla funzione `reviews` una lista non vuota, i nuovi elementi saranno aggiunti in coda senza sovrascrivere quelli già esistenti.

In [10]:
review_list = [] 
score_list = []
date_list = []
title_list = []


In [40]:
url = 'https://www.imdb.com/search/title/?title_type=feature&release_date=2001-01-01,2001-12-31&genres=drama&count=250'
(review, score, date, title) = reviews(url=url, 
                                       review_list=review_list, 
                                       score_list=score_list, 
                                       date_list=date_list, 
                                       title_list=title_list)

<font size=3>

Come detto in precedenza il processo per ricavare le recensioni di quasi $30$ anni ha richiesto diverso tempo e diverse sessioni di lavoro. Una volta chiuso il notebook le variabili presenti sono eliminate, quindi sono stati creati dei salvataggi ad ogni sessione attraverso la funzione `pickle.dump` che crea un oggetto con estensione anonima in cui gli elementi sono salvati senza modifiche e possono essere ricaricati all'inizio di ogni sessione senza ulteriori modifiche.

<br></br>
## [1.2 Pickle Save](#1.2)

In [41]:
os.chdir('D:/Scraping')
with open('output_review', 'wb') as fp:
    pickle.dump(review_list, fp)
    
with open('output_score', 'wb') as fp2:
    pickle.dump(score_list, fp2)
    
with open('output_date', 'wb') as fp3:
    pickle.dump(date_list, fp3)
    
with open('output_title', 'wb') as fp4:
    pickle.dump(title_list, fp4)

<br></br>
<br></br>
# [2. Controllo Recensioni](#2)
<br></br>
## [2.1 Confronto lunghezza e titoli](#2.1)

<font size=3>
    
Si è notato che c'erano alcuni problemi con particolari titoli di film, nello specifico quelli relativi al $2016$ e $2017$ per la presenza di parentesi all'interno del titolo (parentesi non relative all'anno di uscita). Questo codice ha cercato di sistemare queste problematiche e successivamente viene effettuato un conteggio per verificare se tutti i film sono stati presi una volta sola. <br></br>

Il print è stato effettuato dopo la correzione nei blocchi successivi, in origine un titolo era presente due volte.

In [144]:
film = []
for el in title_list:
    temp = el.split('(')
    if len(temp) == 2:
        if '2017' or '2016' in temp[1]:
            film.append(temp[0] + '(' + temp[1])
    if len(temp) == 3:
        if '2017' or '2016' in temp[2]:
            film.append(temp[0] + ' ' + temp[1] + '(' + temp[2])

freq = Counter(film).most_common(10)
freq

[("Le Mans '66 - La grande sfida (2019)", 1),
 ('Joker (2019)', 1),
 ('The Irishman (2019)', 1),
 ('Cats (2019)', 1),
 ('Doctor Sleep (2019)', 1),
 ('Midway (2019)', 1),
 ("C'era una volta a... Hollywood (2019)", 1),
 ('Cena con delitto - Knives Out (2019)', 1),
 ('Last Christmas (2019)', 1),
 ('Il re  I) (2019)', 1)]

<font size=3>
    
Un titolo risulta duplicato e non ci sono recensioni a cui fa riferimento, viene quindi eliminato l'elemento dalla lista.

In [ ]:
print(title_list.index(b[0][0]))
print(title_list[501])
print(review_list[500], '\n', review_list[501])
print(title_list[502])

del title_list[501]

<font size=3>

Stranamente era presente un titolo duplicato a cui però non erano legate recensioni, quindi probabilmente il codice di scraping si era fermato ad individuare solamente il primo titolo di un film e poi per qualche motivo si è interrotto.
Da qui la necessità di sistemare l'ordine dei film presenti nella lista completa:
* Il primo for entra all'interno di ogni gruppo di recensioni
* Il secondo for aggiunge nella nuova lista il nome del film di riferimento tante volte quanti sono i commenti ottenuti.

In [6]:
new_title_list = []
for cont in range(len(review_list)):
    for cont2 in range(len(review_list[cont])):
        new_title_list.append(title_list[cont])

<br></br>
## [2.2 Confronto score e data](#2.2)

<font size=3>

Analogamente a quanto fatto in precedenza si controlla la lunghezza delle liste contenenti il punteggio e la data della recensione. Dato che si lavora con liste di liste, si controlla la lunghezza di tutte le lunghezze delle liste interne, se la lunghezza delle due liste non combacia significa che in una delle due è stato salvato un elemento in più o in meno. Il codice stampa a video l'indice della lista per cui si verificano problemi.

In [128]:
for i in range(len(score_list)):
    if len(review_list[i]) == len(date_list[i]):
        pass
    else:
        print(i)

540


<font size=3>

Per un errore del sito, una data non è visualizzata correttamente, come se non ci fosse una tabulazione fra il nome dell'utente che ha postato la recensione e la data, in questo modo i due oggetti appaiono attaccati come rappresentato nella seguente immagine nel riquadro nero.
![title](Date_error1.png) <br></br>

Questa struttura diversificata non permette l'individuazione corretta dell'xpath associato alla data, mentre per il punteggio non ci sono problemi, da qui la discrepanza nelle lunghezze delle liste. <br></br>

La data con errore è stata individuata manualmente non avendo a disposizioni altre informazioni se non il titolo del film per identificare l'errore. Sempre manualmente quindi si procede con l'aggiungere la data mancante alla lista:
* Attraverso la funzione `index` viene identificato l'indice della data specificata, in questo caso è l'indice relativo alla prima data ottenuta dopo quella dove si è verificato l'errore.
* Si salva in una variabile temporanea il contenuto della lista delle date fino all'indice identificato al passo prima e lo stesso contenuto viene eliminato dalla lista originale. <br></br> 
(nota: questo meccanismo è rischioso in quanto la funzione `index` restituisce il primo indice di cui trova l'elemento, nel caso ci fossero state più recensioni nella stessa data, sarebbe potuto accadere che l'indice in output non corrispondesse a quello dopo la data mancante. La possibilità di questo errore è presente in quanto le recensioni sul sito sono elencate in base all'"utilità" e non per data. L'utilità è il rapporto fra il numero di persone che ha ritenuto quella recensione utile e quelli che non l'hanno trovata utile, in caso di rapporto uguale prevale il numero di recensioni utili e successivamente si procede per data. Nel caso in questione solo una recensione è stata scritta in quella data quindi l'indice riportato è certamente quello corretto).
* Viene aggiunta la data mancante alla data originale e successivamente il contenuto della lista temporanea che è stato momentaneamente eliminato.

In [7]:
idx = date_list[540].index("25 February 2019")
temp = date_list[540][idx:]
del date_list[540][idx:]

date_list[540].append("31 March 2019")
for el in temp:
    date_list[540].append(el)

<font size=3>

La funzione `deepflatten` restituisce un oggetto in cui tutti gli elementi al suo interno sono posti ad una determinata stratificazione, specificando `depth=1` sono posti tutti allo stesso livello. L'oggetto risultante viene poi trasfomrato in una lista. <br></br>
Questo passaggio deriva dalla necessità di dover ottenere un'unica lista con le recensioni, punteggio e data e non una lista di liste.

In [8]:
reivew_flatten = list(deepflatten(review_list, depth=1))
score_flatten = list(deepflatten(score_list, depth=1))
date_flatten = list(deepflatten(date_list, depth=1))

<font size=3>

Partendo dalla lista di film modificata eliminando il doppione e dalle liste di liste rese una lista unica, si crea un dataftame contenente queste informazioni.

In [9]:
df = pd.DataFrame({'Title':new_title_list, 'Review':reivew_flatten, 'Date':date_flatten, 'Score':score_flatten})
df.head()

,Date,Review,Score,Title
0,16 November 2019,This is what cinema is supposed to be! Amazing...,10,Le Mans '66 - La grande sfida (2019)
1,20 November 2019,Rating 10/10 Inspired movie of the year. It is...,10,Le Mans '66 - La grande sfida (2019)
2,15 November 2019,I was at the Philadelphia Film center premiere...,10,Le Mans '66 - La grande sfida (2019)
3,31 August 2019,Just saw F v F at the Telluride Film Festival ...,10,Le Mans '66 - La grande sfida (2019)
4,10 September 2019,Biopics can be a hard sell at times. Whether y...,9,Le Mans '66 - La grande sfida (2019)


<br></br>
<br></br>
# [3 Merge](#3)
<br></br>
## [3.1 Part 2](#3.1)

<font size=3>

Come già detto in precedenza lo scraping ha richiesto molto tempo e per velocizzare il processo si è provato a utilizzare la funzione `review` su più notebook in quanto in uno stesso non è possibile far girare più blocchi di codice contemporaneamente. <br></br>
Per gli altri notebook il contenuto era lo stesso qua presente e alla fine di ogni sessione veniva salvato l'output delle liste. Una volta terminato il processo i file completi sono caricati tramite la libreria `pickle` come visto in precedenza. In seguito ci sono tutti gli step di controllo sui titoli e le recensioni che sono stati applicati in precedenza all'output di questo notebook.

In [10]:
with open('output_review2', "rb") as input_file:
    review_list2 = pickle.load(input_file)
with open('output_score2', "rb") as input_file2:
    score_list2 = pickle.load(input_file2)
with open('output_date2', "rb") as input_file3:
    date_list2 = pickle.load(input_file3)
with open('output_title2', "rb") as input_file4:
    title_list2 = pickle.load(input_file4)

In [159]:
print(len(review_list2), len(score_list2), len(date_list2), len(title_list2))

for i in range(len(review_list2)):
    if len(review_list2[i]) == len(date_list2[i]):
        pass
    else:
        print(i)

1500 1500 1500 1500


<font size=3>

Originariamente c'era una recensione vuota in ecccesso.

In [11]:
del review_list2[1204][2]

<font size=3>

I tre step vist in in precedenza sono combinati:
* Creazione di una lista unica partendo da liste di liste
* Controllo dei titoli dei film
* Creazione di un dataframe

In [12]:
reivew_flatten2 = list(deepflatten(review_list2, depth=1))
score_flatten2 = list(deepflatten(score_list2, depth=1))
date_flatten2 = list(deepflatten(date_list2, depth=1))

new_title_list2 = []
for cont in range(len(review_list2)):
    for cont2 in range(len(review_list2[cont])):
        new_title_list2.append(title_list2[cont])
        
df2 = pd.DataFrame({'Title':new_title_list2, 'Review':reivew_flatten2, 'Date':date_flatten2, 'Score':score_flatten2})
df2.head()

,Date,Review,Score,Title
0,23 December 2012,I only had one thought on my mind for this Chr...,8,Django Unchained (2012)
1,26 December 2012,Merry Christmas to all you Tarantino fans out ...,10,Django Unchained (2012)
2,26 December 2012,Absolutely loved every minute of this movie. U...,10,Django Unchained (2012)
3,26 December 2012,"Quentin Tarantino's 8th film ""Django Unchained...",10,Django Unchained (2012)
4,1 January 2013,"In Quentin Tarantino's Django Unchained, there...",9,Django Unchained (2012)


<br></br>
## [3.2 Part 3](#3.2)

In [13]:
with open('output_review3', "rb") as input_file:
    review_list3 = pickle.load(input_file)
with open('output_score3', "rb") as input_file2:
    score_list3 = pickle.load(input_file2)
with open('output_date3', "rb") as input_file3:
    date_list3 = pickle.load(input_file3)
with open('output_title3', "rb") as input_file4:
    title_list3 = pickle.load(input_file4)

In [173]:
print(len(review_list3), len(score_list3), len(date_list3), len(title_list3))

for i in range(len(review_list3)):
    if len(review_list3[i]) == len(score_list3[i]):
        pass
    else:
        print(i)

1500 1500 1500 1500


<font size=3>
    
Anche in questo caso si aveva una recensione vuota extra.

In [14]:
del review_list3[971][1]

In [15]:
reivew_flatten3 = list(deepflatten(review_list3, depth=1))
score_flatten3 = list(deepflatten(score_list3, depth=1))
date_flatten3 = list(deepflatten(date_list3, depth=1))

new_title_list3 = []
fl = 0
for cont in range(len(review_list3)):
    for cont2 in range(len(review_list3[cont])):
        new_title_list3.append(title_list3[cont])
        
df3 = pd.DataFrame({'Title':new_title_list3, 'Review':reivew_flatten3, 'Date':date_flatten3, 'Score':score_flatten3})
df3.head()

,Date,Review,Score,Title
0,8 March 2009,"For over 25 years now, I have cited Blade Runn...",10,Watchmen (2009)
1,25 February 2009,Screened FRebruary 23 for Australian Media.\n\...,10,Watchmen (2009)
2,3 March 2009,"Before anyone sees this film, Zach Snyder shou...",8,Watchmen (2009)
3,14 March 2009,Watchmen is the long-awaited graphic novel ada...,10,Watchmen (2009)
4,10 March 2009,"Firstly, I have not read the graphic novel. Th...",10,Watchmen (2009)


<br></br>
## [3.3 Part 4](#3.3)

In [16]:
with open('output_review4', "rb") as input_file:
    review_list4 = pickle.load(input_file)
with open('output_score4', "rb") as input_file2:
    score_list4 = pickle.load(input_file2)
with open('output_date4', "rb") as input_file3:
    date_list4 = pickle.load(input_file3)
with open('output_title4', "rb") as input_file4:
    title_list4 = pickle.load(input_file4)

In [184]:
print(len(review_list4), len(score_list4), len(date_list4), len(title_list4))

for i in range(len(review_list4)):
    if len(review_list4[i]) == len(date_list4[i]):
        pass
    else:
        print(i)

1500 1500 1500 1500
1011


<font size=3>

Anche in questo caso la differente lunghezza delle liste è dovuto al fatto che una data non è stata catturata correttamente. Per risolvere il problema si procede in maniera analoga a quanto visto in precedenza. Le prime righe di codice servono per avere un'idea di dove si trovi il commento con errore.
![title](Date_error2.png) <br></br>

In [17]:
temp_l = []
for i in range(len(date_list4[1011])):
    temp_l.append(score_list4[1011][i] + ' - ' + date_list4[1011][i])

date_list4[1011].index("25 January 2005")


temp = date_list4[1011][129:]
del date_list4[1011][129:]
date_list4[1011].append("29 June 2004")
for el in temp:
    date_list4[1011].append(el)

In [18]:
reivew_flatten4 = list(deepflatten(review_list4, depth=1))
score_flatten4 = list(deepflatten(score_list4, depth=1))
date_flatten4 = list(deepflatten(date_list4, depth=1))

new_title_list4 = []
fl = 0
for cont in range(len(review_list4)):
    for cont2 in range(len(review_list4[cont])):
        new_title_list4.append(title_list4[cont])
        
df4 = pd.DataFrame({'Title':new_title_list4, 'Review':reivew_flatten4, 'Date':date_flatten4, 'Score':score_flatten4})
df4.head()

,Date,Review,Score,Title
0,9 March 2019,The Departed is one of the rare cases when the...,9,The Departed - Il bene e il male (2006)
1,2 October 2006,Now I know that 'The Departed' is based off of...,10,The Departed - Il bene e il male (2006)
2,26 June 2019,When veteran director Martin Scorsese walked u...,9,The Departed - Il bene e il male (2006)
3,3 October 2006,Please don't make negative comments like some ...,10,The Departed - Il bene e il male (2006)
4,7 October 2006,Just came back from watching the movie so it's...,5,The Departed - Il bene e il male (2006)


<br></br>
## [3.4 Concat](#3.4)

<font size=3>

Creati i singoli dataset si procede con l'unione. In pandas è presente la funzione `concat` che permette di unire più dataset diversi, il comendo `axis=0` indica che devono essere aggiunte delle righe.

In [19]:
imdb_df = pd.concat([df, df2, df3, df4], axis=0)

,Date,Review,Score,Title
0,16 November 2019,This is what cinema is supposed to be! Amazing...,10,Le Mans '66 - La grande sfida (2019)
1,20 November 2019,Rating 10/10 Inspired movie of the year. It is...,10,Le Mans '66 - La grande sfida (2019)
2,15 November 2019,I was at the Philadelphia Film center premiere...,10,Le Mans '66 - La grande sfida (2019)
3,31 August 2019,Just saw F v F at the Telluride Film Festival ...,10,Le Mans '66 - La grande sfida (2019)
4,10 September 2019,Biopics can be a hard sell at times. Whether y...,9,Le Mans '66 - La grande sfida (2019)


<br></br>
## [3.5 File Save/Read](#3.5)

In [20]:
#imdb_df.to_csv('imdb_df.csv', sep='\t')
imdb_df = pd.read_csv("imdb_df.csv", delimiter='\t')

<br></br>
<br></br>
# [4. Scraping Titoli](#4)    

<font size=3>

In un secondo momento si è notato che il sito applica una geolocalizzazione automatica che non è modificabile per gli ospiti, ma si dovrebbe fare un login e cambiare la lingua nelle impostazioni. Questa seconda soluzioni non è percorribile in quanto il login tramite un browser gestito da un robot è vietato. <br></br>
Nelle pagine con la lista dei film i titoli erano riportati con la geolocalizzazione e questo è stato il processo con cui sono stati salvati. L'alternativa è aprire la pagina principale del film e all'inizio è presente sia il titolo geolocalizzato sia il titolo originale. <br></br>

Per ricavare i titoli originali viene creata una nuova funzione `movie_info2` che riprende per alcuni tratti quello già visto nella funzione `reviews`. I parametri in ingresso sono:
* `url`: in questo caso, come verrà specificato in seguito, non si tratta di un url singolo ma di una lista di indirizzi, dove l'unica differnza è l'anno di riferimento. L'estrazione del titolo originale è una procedura che è molto più veloce rispetto all'identificazione e il salvataggio di centinaia di recensioni per un singolo film, quindi risulta fattibile l'utilizzo di una lista.
* `title_list`: è la lista in cui sono salvati i titoli geolocalizzati, già ottenuti con la funzione precedente. Questi valori fungeranno da chiave il successivo merge.
* `original_title_list`: come dice il nome è la lista in cui saranno salvati i titoli originali dei film.

Il codice inizia con l'inizializzazione di un contatore `j` che servirà in seguito. Le operazioni svolte dalla funzione sono:
* Prende volta per volta un url della lista in ingresso, viene impostato il browser per evitare di caricare le immagini nelle pagine e si apre una pagina sull'url inserito.
* Si crea una lista in cui saranno inseriti i collegamenti ipertestuali che rimandano alla pagina principale di ogni film. Il ciclo for itera per tutti i $250$ film della pagina estraendo il nome (geolocalizzato) in modo identico a quello che era stato fatto in precedenza. Allo stesso modo vengono estratti gli url dei film ed inseriti nella lista.
* Per questa procedura risulta più comodo aprire una nuova finestra che sarà chiusa solo alla fine delle operazioni e non volta per volta in quanto le informazioni da scaricare sono inferiori a quelle della prima funzione. Per ogni indirizzo ipertestuale si ottiene l'url per accedere alla pagina principale del film.
* Aperta la pagina principale ci possono essere due casi: il titolo è effettivamente geolocalizzato, oppure il titolo non è stato tradotto ma (in questo caso in Italia) si utilizza sempre il titolo originale. 
    * Se il titolo è stato tradotto,allora è presente sotto al titolo geolocalizzato il titolo originale con l'aggiunta di '(original title)'
    * Se il titolo non è stato tradotto, allora è presente un unico titolo.
* Mediante la funzione `find_elements_by_partial_link_text` è possibile indicare al browser di identificare un oggetto nella pagina in cui compare un determinato testo, in questo caso quello che identifica la presenza di un titolo originale che differisce da quello geolocalizzato, ovvero '(original title)'. Se questo elemento non è presente si tratta quindi di un film il cui titolo non è stato tradotto. In caso contrario si procede con l'identifcazione dell'xpath di esso.
* Nella lista dei titoli tradotto si aggiunge l'elemento trovato. Nel caso in cui il titolo non sia tradotto allora si deve far riferimento alla lista dei titoli ottenuti all'inizio del codice. A differenza della funzione `reviews` qua si effettua un ciclo for su diversi url, da qui l'esigenza di utilizzare un ulteriore contatore `j` per contare le iterazioni degli url e posizionare i titoli non tradotti correttamente nella lista.

Una volta ottenuti i titoli di un determinato anno i due browser vengono chiusi e si procede con l'url successivo. La funzione ritorna come output la lista dei titoli originali e quelli tradotti.

In [23]:
def movie_info2(url, title_list, original_title_list):
    j = 0
    for url in url_list:
        firefox_profile = webdriver.FirefoxProfile()
        firefox_profile.set_preference('permissions.default.image', 2)
        firefox_profile.set_preference('dom.ipc.plugins.enabled.libflashplayer.so', 'false')
        movie = webdriver.Firefox(firefox_profile=firefox_profile, executable_path=r'D:/Scraping2/geckodriver.exe')
        movie.get(url)

        href_list = []
        for i in range(250):
            title = movie.find_element_by_xpath('/html/body/div[3]/div/div[2]/div[3]/div[1]/div/div[3]/div/div[' + str(i + 1) + ']/div[3]/h3/a')
            title2 = movie.find_element_by_xpath('/html/body/div[3]/div/div[2]/div[3]/div[1]/div/div[3]/div/div[' + str(i + 1) + ']/div[3]/h3/span[2]')

            title_list.append(title.text + ' ' + title2.text)

            xpath = '//*[@id="main"]/div/div[3]/div/div[' + str(i + 1) + ']/div[3]/h3/a'
            element = movie.find_element_by_xpath(xpath=xpath)
            href = element.get_attribute('href')
            href_list.append(href)

        movie2 = webdriver.Firefox(firefox_profile=firefox_profile, executable_path=r'D:/Scraping2/geckodriver.exe')
        for i in range(250):
            temp_url = href_list[i].split('?')
            movie2.get(temp_url[0])

            try:
                movie2.find_elements_by_partial_link_text("(original title)")
                original_title = movie2.find_element_by_xpath("/html/body/div[3]/div/div[2]/div[5]/div[1]/div/div/div[1]/div[2]/div/div[2]/div[2]/div[1]")
                if ("(original title)") not in original_title.text:
                    original_title_list.append(title_list[(i + 250*j)])
                else:
                    original_title_list.append(original_title.text)
            except (NoSuchElementException, ElementNotInteractableException):
                original_title_list.append("")
        j = j + 1
        movie2.quit()
        movie.quit()
    
    return(title_list, original_title_list)

<font size=3>

Per creare gli url si fa riferimento a:
* Un url di base
* L'anno che viene creato tramite un ciclo for decrescente, partendo dall'anno più recente a quello più lontano. L'anno poi viene attaccato come stringa al mese e anno di inizio e fine ricerca che sono sempre fissi a $1$ gennaio e $31$ dicembre.
* L'ultima parte dell'url è relativa al genere e al numero di film da visualizzare per anno.

In [22]:
url = 'https://www.imdb.com/search/title/?title_type=feature&release_date='
url_list = []
for i in range(2019, 2011, -1):
    url_list.append(url + str(i) + "-01-01," + str(i) + "-12-31&genres=drama&count=250")
print(url_list)

['https://www.imdb.com/search/title/?title_type=feature&release_date=2019-01-01,2019-12-31&genres=drama&count=250', 'https://www.imdb.com/search/title/?title_type=feature&release_date=2018-01-01,2018-12-31&genres=drama&count=250', 'https://www.imdb.com/search/title/?title_type=feature&release_date=2017-01-01,2017-12-31&genres=drama&count=250', 'https://www.imdb.com/search/title/?title_type=feature&release_date=2016-01-01,2016-12-31&genres=drama&count=250', 'https://www.imdb.com/search/title/?title_type=feature&release_date=2015-01-01,2015-12-31&genres=drama&count=250', 'https://www.imdb.com/search/title/?title_type=feature&release_date=2014-01-01,2014-12-31&genres=drama&count=250', 'https://www.imdb.com/search/title/?title_type=feature&release_date=2013-01-01,2013-12-31&genres=drama&count=250', 'https://www.imdb.com/search/title/?title_type=feature&release_date=2012-01-01,2012-12-31&genres=drama&count=250']


In [24]:
title_list = []
original_title_list = []

<font size=3>
    
Poiché si effettua un'iterazione su diversi url non è necessario richiamare più volte la funzione. <br></br>
Durante la ricerca degli url si sono riscontrati dei problemi relativi a dei file locali, molto probabilmente legato al fatto che si è imposto al browser di non caricare le immagini e questo codice si ripeteva ogni volta che si passava ad un nuovo url. Nonostante gli errori mostrati a video il codice non si è interrotto, sono da interndesi più come dei warning, e i titoli sono stati salvati nelle liste senza problemi.

In [25]:
(title, original_title) = movie_info2(url=url_list,
                                      title_list=title_list,
                                      original_title_list=original_title_list)

[WinError 3] Impossibile trovare il percorso specificato: 'C:\\Users\\Hp\\AppData\\Local\\Temp\\tmpul9157do'
[WinError 3] Impossibile trovare il percorso specificato: 'C:\\Users\\Hp\\AppData\\Local\\Temp\\tmpmf_gnd5f'
[WinError 3] Impossibile trovare il percorso specificato: 'C:\\Users\\Hp\\AppData\\Local\\Temp\\tmp7tb9d8j6'
[WinError 3] Impossibile trovare il percorso specificato: 'C:\\Users\\Hp\\AppData\\Local\\Temp\\tmpiaya8g39'
[WinError 3] Impossibile trovare il percorso specificato: 'C:\\Users\\Hp\\AppData\\Local\\Temp\\tmp_ijssw4r'
[WinError 3] Impossibile trovare il percorso specificato: 'C:\\Users\\Hp\\AppData\\Local\\Temp\\tmpng1n6i3_'
[WinError 3] Impossibile trovare il percorso specificato: 'C:\\Users\\Hp\\AppData\\Local\\Temp\\tmp9lxrews9'
[WinError 3] Impossibile trovare il percorso specificato: 'C:\\Users\\Hp\\AppData\\Local\\Temp\\tmpvvp191iu'


In [27]:
for el in title:
    print(el)

The Irishman (2019)
Cena con delitto - Knives Out (2019)
C'era una volta a... Hollywood (2019)
Le Mans '66 - La grande sfida (2019)
Joker (2019)
La ragazze di Wall Street (2019)
Un amico straordinario (2019)
Parasite (2019)
Midway (2019)
Doctor Sleep (2019)
Last Christmas (2019)
Honey Boy (2019)
1917 (2019)
21 Bridges (2019)
Un cavaliere per Natale (2019)
It - Capitolo due (2019)
Il re (I) (2019)
Blue Story (2019)
Il Re Leone (2019)
The Lighthouse (I) (2019)
Queen & Slim (2019)
Avengers: Endgame (2019)
Cats (2019)
Gemini Man (2019)
Lilli e il vagabondo (2019)
Jojo Rabbit (2019)
Piccole donne (2019)
The Peanut Butter Falcon (2019)
Storia di un matrimonio (2019)
Richard Jewell (2019)
The Report (I) (2019)
Midsommar - Il villaggio dei dannati (I) (2019)
Downton Abbey (2019)
L'inganno perfetto (2019)
The Courier (2019)
Brittany non si ferma più (2019)
Bombshell (2019)
Dark Waters (2019)
Dove la terra trema (2019)
Il cardellino (2019)
Panama Papers (I) (2019)
Motherless Brooklyn - I segreti

In [28]:
for el in original_title:
    print(el)

The Irishman (2019)
Knives Out (original title)
Once Upon a Time... in Hollywood (original title)
Ford v Ferrari (original title)
Joker (2019)
Hustlers (original title)
A Beautiful Day in the Neighborhood (original title)
Gisaengchung (original title)
Midway (2019)
Doctor Sleep (2019)
Last Christmas (2019)
Honey Boy (2019)
1917 (2019)
21 Bridges (2019)
The Knight Before Christmas (original title)
It Chapter Two (original title)
The King (original title)
Blue Story (2019)
The Lion King (original title)
The Lighthouse (I) (2019)
Queen & Slim (2019)
Avengers: Endgame (2019)

Gemini Man (2019)
Lady and the Tramp (original title)
Jojo Rabbit (2019)

The Peanut Butter Falcon (2019)
Marriage Story (original title)
Richard Jewell (2019)
The Report (I) (2019)
Midsommar (original title)
Downton Abbey (2019)
The Good Liar (original title)
The Courier (2019)
Brittany Runs a Marathon (original title)

Dark Waters (2019)
Earthquake Bird (original title)
The Goldfinch (original title)
The Laundromat 

<br></br>
## [4.1 Merge Titoli](#4.1)

<font size=3>
    
Le liste ottenute con gli altri notebook sono stati salvati in file `picle` e caricate nel seguente blocco di codice. Per facilitare la creazione di una lista unica i file sono inseriti in due liste: una per i titoli geolocalizzati ed una per i titoli originali.

In [ ]:
with open('output_title2', "rb") as input_file:
    title2 = pickle.load(input_file)
with open('output_original_title2', "rb") as input_file2:
    original_title2 = pickle.load(input_file2)
with open('output_title3', "rb") as input_file3:
    title3 = pickle.load(input_file4)
with open('output_original_title3', "rb") as input_file4:
    original_title3 = pickle.load(input_file4)
with open('output_title4', "rb") as input_file5:
    title4 = pickle.load(input_file5)
with open('output_original_title4', "rb") as input_file6:
    original_title4 = pickle.load(input_file6)
    
title_files = [title2, title3, title4]
original_title_files = [original_title2, original_title3, original_title4]

<font size=3>
    
Per ogni file all'interno della lista si prende un titolo alla volta e lo si aggiunge alla lista già presente in questo notebook.

In [ ]:
for file in title_files:
    for t in file:
        title.append(t)
        
for file in original_title_files:
    for o in file:
        original_title.append(o)

<font size=3>

Dalle due liste ottenute si crea un dataframe con `pandas` e il risultato viene salvato in un `.csv`.

In [ ]:
movies_titles = pd.DataFrame({"Translated":title, "Original":original_title})
movies_titles.to_csv("movies titles.csv")

<br></br>
<br></br>
# [5. Scraping Info](#5)

<font size=3>

Per effettuare una aspect-base sentiment analysis corretta sono necessarie ulteriori informazioni relative al film. Come letto in diversi paper le informazioni di contorno come il nome degli attori sono fondamentali per individuare correttamente gli aspetti ed effettuare un'analisi del sentiment.

Per estrarre queste informazioni si deve far riferimento ad un'altra pagina e quindi è necessario creare una nuova funzione di scraping, denominata `movie_info`; i parametri in ingresso sono:
* `url`: è lo stesso url utilizzato nelle altre funzioni, in questo caso non verrà iterato.
* `director_list`: in questa lista verranno salvati i nomi dei direttori.
* `writer_list`: sono salvati i nomi degli scrittori dei copioni.
* `actor_list`: sono salvati i nomi degli attori.
* `fict_actor_list`: sono salvati i nomi dei personaggi interpretati dagli attori.
* `title_list`: anche questa volta il nome del film sarà usato come chiave per un futuro merge.

Il codice inizia con la creazione di un oggetto browser e l'accesso alla lista dei $250$ film più popolari di un certo anno. I successivi step sono:
* Il salvataggio del titolo del film, identico a quanto effettuato in precedenza. 
* Si estraggono i collegamenti ipertestuali di ogni film, da essi si modifica l'url in modo tale da accedere alla pagina con le info delle personalità legate al film. 
* Sono inizializzate le liste temporanee in cui saranno salvati gli elementi e alla fine di un'iterazione esse saranno aggiunte alle liste passate in input.
* Per ogni film si apre una nuova finestra e in questa si cercano le informazioni necessarie sempre tramite l'`xpath`. La lunghezza delle iterazioni dipende dalle informazioni da ricercare, ad esempio per gli attori si cercheranno tanti elementi, per i/il direttori/e meno.

Gli output sono delle liste in cui ogni elemento è una lista contenente le personalità riferite ad una certa categoria.

In [149]:
def movie_info(url, director_list, writer_list, actor_list, fict_actor_list, title_list):
    movie = webdriver.Firefox(executable_path=r'D:/Scraping2/geckodriver.exe')
    movie.get(url)
    
    for i in range(250):
        title = movie.find_element_by_xpath('/html/body/div[3]/div/div[2]/div[3]/div[1]/div/div[3]/div/div[' + str(i + 1) + ']/div[3]/h3/a')
        title2 = movie.find_element_by_xpath('/html/body/div[3]/div/div[2]/div[3]/div[1]/div/div[3]/div/div[' + str(i + 1) + ']/div[3]/h3/span[2]')
        
        title_list.append(title.text + ' ' + title2.text)
        
        xpath = '//*[@id="main"]/div/div[3]/div/div[' + str(i + 1) + ']/div[3]/h3/a'
        element = movie.find_element_by_xpath(xpath=xpath)
        href = element.get_attribute('href')
    
        temp_url = href.split('?')
        credits_url = temp_url[0] + "fullcredits?ref_=tt_ov_wr#writers/"
        
        temp_director_list = [] 
        temp_writer_list = []
        temp_actor_list = []
        temp_fict_actor_list = []
        
        movie2 = webdriver.Firefox(executable_path=r'D:/Scraping2/geckodriver.exe')
        movie2.get(credits_url)
        
        # direttore
        try:
            director = movie2.find_element_by_xpath("/html/body/div[3]/div/div[2]/div[3]/div[1]/div[1]/div[2]/table[1]/tbody/tr/td[1]/a")
            temp_director_list.append(director.text)
        except (NoSuchElementException, ElementNotInteractableException):
                temp_director_list.append("")
        
        # scrittori
        for i in range(5):
            try:
                writer = movie2.find_element_by_xpath("/html/body/div[3]/div/div[2]/div[3]/div[1]/div[1]/div[2]/table[2]/tbody/tr[" + str(i + 1) + "]/td[1]/a")
                temp_writer_list.append(writer.text)
            except (NoSuchElementException, ElementNotInteractableException):
                pass
            
        if len(writer_list) == 0:
            temp_writer_list.append("")
        
        # attori
        for i in range(10):
            try:
                actor = movie2.find_element_by_xpath("/html/body/div[3]/div/div[2]/div[3]/div[1]/div[1]/div[2]/table[3]/tbody/tr[" + str(i + 1) + "]/td[2]/a")
                fict_actor = movie2.find_element_by_xpath("/html/body/div[3]/div/div[2]/div[3]/div[1]/div[1]/div[2]/table[3]/tbody/tr[" + str(i + 1) + "]/td[4]/a")
                temp_actor_list.append(actor.text)
                temp_fict_actor_list.append(fict_actor.text)
            
            except (NoSuchElementException, ElementNotInteractableException):
                pass
        if len(actor_list) == 0:
            temp_actor_list.append("")
        if len(fict_actor_list) == 0:
            temp_fict_actor_list.append("")
            
        director_list.append(temp_director_list)
        writer_list.append(temp_writer_list)
        actor_list.append(temp_actor_list)
        fict_actor_list.append(temp_fict_actor_list)
        
        movie2.quit()
        
    movie.quit()
    
    return(director_list, writer_list, actor_list, fict_actor_list, title_list)

In [150]:
director_list = []
writer_list = []
actor_list = []
fict_actor_list = []
title_list = []

In [151]:
url = 'https://www.imdb.com/search/title/?title_type=feature&release_date=1995-01-01,1995-12-31&genres=drama&count=250'
(director, writer, actor, fict_actor, title) = movie_info(url,
                                                          director_list=director_list,
                                                          writer_list=writer_list,
                                                          actor_list=actor_list,
                                                          fict_actor_list=fict_actor_list,
                                                          title_list=title_list)

<font size=3>
    
Sempre tramite `piclke` le liste sono salvate per poi essere caricate successivamente. Nonostante il compito di estrarre queste informazioni non richiedesse un tempo eccessivo, il codice è stato fatto girare su più notebook differenti e quindi per ottenere delle liste complete si dovranno caricare gli altri file e unirli assieme.

In [36]:
os.chdir('D:/Scraping')
with open('output_director', 'wb') as fp:
    pickle.dump(director_list, fp)
    
with open('output_writer', 'wb') as fp2:
    pickle.dump(writer_list, fp2)
    
with open('output_actor', 'wb') as fp3:
    pickle.dump(actor_list, fp3)
    
with open('output_fict_actor', 'wb') as fp4:
    pickle.dump(fict_actor_list, fp4)
    
with open('output_title', 'wb') as fp5:
    pickle.dump(title_list, fp5)

<br></br>
## [5.1 Merge Info](#5.1)

<font size=3>

Lo scraping delle info ha funzionato a dovere anche se in alcuni casi se l'attore di un film non possiede una pagina personale sul sito, la formattazione risulta divera e l'algoritmo non è stato in grado di ottenere le informazioni. Si tratta per lo più di attori poco conosciuto e che hanno recitato in film non molto popolari, quindi i dati mancanti sono comunque ristretti. <br></br>
Per due anni, il $2019$ e il $1995$, il codice ha dato problemi con il risultato di avere delle informazioni duplicate:
* Per il $2019$ i doppioni riguardavano le prime osservazioni. Si è trovata questa anomalia confrontando la lunghezza di tutte le liste caricate e si sono subito trovati questi duplicati all'inizio della prima lista. Per tutte le informazioni le prime $24$ osservazioni erano ridondanti con l'eccezione dei titoli dove erano $25$.
* Per il $1995$ la situazione era più complicata in quanto non c'era un pattern preciso che permettesse di individuare facilmente i duplicati, quindi si è proceduto a raccogliere di nuovo tutte le informazioni per quell'anno.

Il caricamento dei file avviene come al solito tramite `pickle`, per vitare un codice lungo e ripetitivo si è provveduto a creare un ciclo in modo tale da poter leggere assieme tutte le liste di una certa categoria:
* Gli indici sono stati messi in una lista, i file sono stati salvati in modo tale da variare solamente nell'ultimo carattere che rappresenta appunto il numero del file salvato, con l'eccezione del $1995$ come spiegato sopra.
* Mediante la funzione `globals()` è possibile creare variabili all'interno del ciclo for con un nome comune e che variano per un elemento finale, in questo caso proprio l'indice del for che permette di accedere correttamente ai file.
* Nel caso della prima lista viene effettuato uno slicing per rimuovere i duplicati.

I risultati sono salvati in delle liste differenziate create in partenza.

In [284]:
director_files, writer_files, actor_files, fict_actor_files, title_files = ([] for i in range(5))
idx = [1, 2, 3, 4, 1995]
for i in idx:
    if i == 1:
        i = ""
    with open('output_director' + str(i), "rb") as director_file:
        globals()["director%s" % (str(i))] = pickle.load(director_file)
        if i == "":
            director = director[23:1523]
    
    with open('output_writer' + str(i), "rb") as writer_file:
        globals()["writer%s" % (str(i))] = pickle.load(writer_file)
        if i == "":
            writer = writer[23:1523]
        
    with open('output_actor' + str(i), "rb") as actor_file:
        globals()["actor%s" % (str(i))] = pickle.load(actor_file)
        if i == "":
            actor = actor[23:1523]
        
    with open('output_fict_actor' + str(i), "rb") as fict_actor_file:
        globals()["fict_actor%s" % (str(i))] = pickle.load(fict_actor_file)
        if i == "":
            fict_actor = fict_actor[23:1523]
        
    with open('output_title' + str(i), "rb") as title_file:
        globals()["title%s" % (str(i))] = pickle.load(title_file)
        if i == "":
            title = title[24:1524]
        
    director_files.append(globals()["director%s" % (str(i))])
    writer_files.append(globals()["writer%s" % (str(i))])
    actor_files.append(globals()["actor%s" % (str(i))])
    fict_actor_files.append(globals()["fict_actor%s" % (str(i))])
    title_files.append(globals()["title%s" % (str(i))])

<font size=3>
    
Un controllo della lunghezza delle liste lette permette di confermare che tutti hanno lo stesso numero di elementi. La somma della prima lista e quella dell'ultima restituisce il valore di lunghezza di quelle intermedie.

In [291]:
for i in range(5):
    print(len(director_files[i]), len(writer_files[i]), len(actor_files[i]), len(fict_actor_files[i]), len(title_files[i]))

1500 1500 1500 1500 1500
1750 1750 1750 1750 1750
1750 1750 1750 1750 1750
1750 1750 1750 1750 1750
250 250 250 250 250


Con la funzione `deepflatten` si ottiene una lista sola per categoria che viene trasformata in una colonna del dataframe. L'ultimo passaggio permette di riordinare le colonne in quanto di default sono messe in ordine alfabetico.

In [295]:
director_flatten = list(deepflatten(director_files, depth=1))
writer_flatten = list(deepflatten(writer_files, depth=1))
actor_flatten = list(deepflatten(actor_files, depth=1))
fict_actor_flatten = list(deepflatten(fict_actor_files, depth=1))
titles_flatten = list(deepflatten(title_files, depth=1))

df_info = pd.DataFrame({'Title':titles_flatten, 'Director':director_flatten, 'Writer':writer_flatten, 
                        'Actor':actor_flatten, 'Fict_actor':fict_actor_flatten})
df_info = df_info[["Title", "Director", "Actor", "Fict_actor", "Writer"]]
df_info.head()

,Title,Director,Actor,Fict_actor,Writer
0,The Irishman (2019),[Martin Scorsese],"[Robert De Niro, Al Pacino, Joe Pesci, Harvey ...","[Frank Sheeran, Jimmy Hoffa, Russell Bufalino,...","[Charles Brandt, Steven Zaillian]"
1,Cena con delitto - Knives Out (2019),[Rian Johnson],"[Daniel Craig, Chris Evans, Ana de Armas, Jami...","[Benoit Blanc, Ransom Drysdale, Marta Cabrera,...",[Rian Johnson]
2,C'era una volta a... Hollywood (2019),[Quentin Tarantino],"[Leonardo DiCaprio, Brad Pitt, Margot Robbie, ...","[Rick Dalton, Cliff Booth, Sharon Tate, Jay Se...",[Quentin Tarantino]
3,Joker (2019),[Todd Phillips],"[Joaquin Phoenix, Robert De Niro, Zazie Beetz,...","[Arthur Fleck, Murray Franklin, Sophie Dumond,...","[Todd Phillips, Scott Silver, Bob Kane, Bill F..."
4,Ad Astra (2019),[James Gray],"[Brad Pitt, Tommy Lee Jones, Ruth Negga, Donal...","[Roy McBride, H. Clifford McBride, Helen Lanto...","[James Gray, Ethan Gross]"


<br></br>
## [5.2 Salvataggio File](#5.2)

In [296]:
df_info.to_csv('imdb_info.csv', sep='\t')
#df_info = pd.read_csv("imdb_info.csv", delimiter='\t')